In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import scipy.optimize as spo
import os
import glob

/home/bumilang/jupyterenv/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
# OPEN_TIME = datetime.strptime('00:00:01', '%H:%M:%S')
# CLOSE_TIME = datetime.strptime('23:59:59', '%H:%M:%S')

OPEN_TIME = '00:00:00'
CLOSE_TIME = '24:00:00'

In [3]:
def get_opt_price_f(f_path):
    opt = pd.read_csv(f_path, encoding = 'CP949');
    opt = opt.rename(columns = {'일자' : 'BASE_DT', '시간' : 'TIME', '선물가격' : 'FUT'});
    
    opt['BASE_DT'] = pd.to_datetime(opt['BASE_DT'], format = '%Y-%m-%d')
    opt['TIME'] = pd.to_datetime(opt['TIME'], unit = 'd').apply(lambda x : (x + timedelta(seconds=0.5)).replace(microsecond = 0)).dt.time
        
    opt = pd.melt(opt, id_vars=['BASE_DT', 'TIME'], var_name= 'INST', value_name='PR');
    fut = opt.loc[opt['INST'] == 'FUT'][['BASE_DT', 'TIME', 'PR']];
    
    opt = opt.loc[(opt['INST'] != 'FUT') & (opt['INST'] < '900')];
    opt = opt.rename(columns = {'INST' : 'STRK'}); 
    opt = opt.astype({'STRK' : 'float'}); 
    
    opt = opt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    fut = fut.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    
    return opt, fut;

In [4]:
def get_opt_price(base_ym, type, dir):
    dir_ym = dir + '/' + str(base_ym)
    
    ohlc = '_C'
    if type.lower() == 'o' :
        ohlc = '_O'
    elif type.lower() == 'h' :
        ohlc = '_H'
    elif type.lower() == 'l' :
        ohlc = '_L'
        
    copt = pd.DataFrame()
    popt = pd.DataFrame()
    fut = pd.DataFrame()
    
    base_dt = set([fn_i[0:8] for fn_i in os.listdir(dir_ym)])
    for base_dt_i in base_dt:
        
        #call option 가격 가져오기
        f_call_i = dir_ym + '/' + base_dt_i + '_Call' + ohlc + '.csv';
        [copt_i, fut_c_i] = get_opt_price_f(f_call_i);
        
        #put option 가격 가져오기
        f_put_i = dir_ym + '/' + base_dt_i + '_Put' + ohlc + '.csv';
        [popt_i, fut_p_i] = get_opt_price_f(f_put_i);        
            
        #선물 가격 재생성
        fut_i = pd.merge(fut_c_i, fut_p_i, how = 'outer', on = ['BASE_DT', 'TIME'], suffixes = ('_C', '_P'));
        fut_i['PR'] = fut_i['PR_C'].fillna(fut_i['PR_P']);
        fut_i.drop(columns = ['PR_C', 'PR_P'], inplace = True);
        
        copt = pd.concat([copt, copt_i])
        popt = pd.concat([popt, popt_i])
        fut = pd.concat([fut, fut_i])
    
    copt = copt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    popt = popt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    fut = fut.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    
    return copt, popt, fut;

In [5]:
def get_opt_atm(base_ym, d_dir, eqvl_srch_rng = 5):
    
    #시가
    [copt_o, popt_o, fut_o] = get_opt_price(base_ym = base_ym, type = 'o', dir = d_dir);
    
    #고가
    [copt_h, popt_h, fut_h] = get_opt_price(base_ym = base_ym, type = 'h', dir = d_dir);
    
    #저가
    [copt_l, popt_l, fut_l] = get_opt_price(base_ym = base_ym, type = 'l', dir = d_dir);
    
    #종가
    [copt_c, popt_c, fut_c] = get_opt_price(base_ym = base_ym, type = 'c', dir = d_dir);
   
    #선물/옵션 데이터 합치기
    fut = pd.merge(fut_c, fut_o, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_O'));
    fut = pd.merge(fut, fut_h, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_H'));
    fut = pd.merge(fut, fut_l, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_L'));
    
    fut = fut.rename(columns = {'PR' : 'FUT', 
                                'PR_O' : 'FUT_O', 
                                'PR_H' : 'FUT_H', 
                                'PR_L' : 'FUT_L', });
    
    copt = pd.merge(copt_c, copt_o, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_O'));
    copt = pd.merge(copt, copt_h, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_H'));
    copt = pd.merge(copt, copt_l, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_L'));
    
    popt = pd.merge(popt_c, popt_o, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_O'));
    popt = pd.merge(popt, popt_h, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_H'));
    popt = pd.merge(popt, popt_l, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_L'));
    
    copt_tmp = copt.rename(columns = {'PR' : 'C_PR', 
                                      'PR_O' : 'C_PR_O', 
                                      'PR_H' : 'C_PR_H', 
                                      'PR_L' : 'C_PR_L', });

    popt_tmp = popt.rename(columns = {'PR' : 'P_PR', 
                                      'PR_O' : 'P_PR_O', 
                                      'PR_H' : 'P_PR_H', 
                                      'PR_L' : 'P_PR_L', });

    opt = pd.merge(copt_tmp, popt_tmp, how = 'inner', on = ['BASE_DT', 'TIME', 'STRK']);
    opt = pd.merge(fut, opt, how = 'left', on = ['BASE_DT', 'TIME'])

    #옵션 우세 찾기
    opt.loc[opt['C_PR'] > opt['P_PR'], 'OPT_WIN'] = 'C';
    opt.loc[opt['C_PR'] < opt['P_PR'], 'OPT_WIN'] = 'P';

    opt = opt.sort_values(by = ['BASE_DT', 'STRK', 'TIME'], ascending=[True, True, True]);
    opt['OPT_WIN'] = opt.groupby(['BASE_DT', 'STRK'])['OPT_WIN'].ffill();
    opt['OPT_WIN_PRV'] = opt.groupby(['BASE_DT', 'STRK'])['OPT_WIN'].shift(1);

    opt_1st_idx = opt.groupby(['BASE_DT', 'STRK']).head(1).index;
    opt.loc[opt_1st_idx, 'OPT_WIN_PRV'] = np.where(opt.loc[opt_1st_idx, 'C_PR_O'] > opt.loc[opt_1st_idx, 'P_PR_O'], 'C', 
                                          np.where(opt.loc[opt_1st_idx, 'C_PR_O'] < opt.loc[opt_1st_idx, 'P_PR_O'], 'P', 'E'))

    #만기 및 만기까지 남은 날
    opt['MATURITY'] = opt['BASE_DT'].max();
    opt['D-Days'] = (pd.to_datetime(opt['MATURITY'], format = '%Y-%m-%d') - pd.to_datetime(opt['BASE_DT'], format = '%Y-%m-%d')).dt.days;
    opt['BizD-Days'] = opt['BASE_DT'].rank(ascending = [True, False], method = 'dense') - 1;

    opt = opt[['BASE_DT', 'MATURITY', 'D-Days', 'BizD-Days', 'TIME', 
               'FUT', 'FUT_O', 'FUT_H', 'FUT_L', 
               'STRK', 'C_PR', 'C_PR_O', 'C_PR_H', 'C_PR_L', 'P_PR', 'P_PR_O', 'P_PR_H', 'P_PR_L', 'OPT_WIN_PRV', 'OPT_WIN']];
    
    opt['EQVL_SUM'] = opt['C_PR'] + opt['P_PR'];

    opt['DIST'] = abs(opt['FUT'] - opt['STRK']);

    #atm 찾기
    atm_srch_rng = 5
    opt_srch = opt[((opt['FUT'] - atm_srch_rng <= opt['STRK']) & (opt['STRK'] <= opt['FUT'] + atm_srch_rng)) & \
                    ((opt['FUT'] - atm_srch_rng <= opt['STRK']) & (opt['STRK'] <= opt['FUT'] + atm_srch_rng))].copy();
    opt_srch = opt_srch.sort_values(['BASE_DT', 'TIME', 'EQVL_SUM', 'DIST', 'STRK'], ascending=[True, True, True, True, False])
    opt_srch['RNK'] = opt_srch.groupby(['BASE_DT','TIME']).cumcount() + 1;
                                
    opt_atm = opt_srch[opt_srch['RNK'] == 1].drop(columns = ['RNK', 'DIST']);

    opt_atm.insert(0, 'BASE_YM', base_ym);
    copt.insert(0, 'BASE_YM', base_ym);
    popt.insert(0, 'BASE_YM', base_ym);
    fut.insert(0, 'BASE_YM', base_ym);
    
    return opt_atm, copt, popt, fut;

In [6]:
def perf_pccrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, pt_dp, pt_buf_dp, opt_atm):

    strg_end_buf_tm = 10*60 + strg_end_buf_tm    #전략 종료 시간 버퍼

    ent_dp = 0.5         # 진입 가격 변동
    clr_dp = 0.5         # 대응 가격 변동

    #atm 자료 시간 오름 차순으로 정렬
    opt_atm = opt_atm.sort_values(by = ['BASE_DT', 'TIME'], ascending=[True, True]);

    #A. 장 시작/종료 시간 추출
    mkt_tm_o = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].head(1);
    mkt_tm_c = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].tail(1);
    mkt_tm = pd.merge(mkt_tm_o, mkt_tm_c, how = 'inner', on = ['BASE_DT'], suffixes=['_O', '_C']);

    strtgy_opt = pd.merge(opt_atm, mkt_tm, how = 'inner', on = ['BASE_DT']);

    #Z. 전략 실행
    # atm_strk_cur
    # win_opt
    #손익
    tot_pl = 0
    fee = 0
    fee_rt = 0.003/100
    exec = pd.DataFrame(columns=['BASE_DT', 'TIME', 'POS_TYPE', 'TR', 'PL', 'FEE'])
    
    #position
    pos = 0
    pos_type = ''

    pr_ent = None # 진입가격
    react_hold_tm = datetime.min
    pr_h = -np.inf
    pr_l = np.inf
    
    for idx, atm_i in strtgy_opt.iterrows():
        tm_i = datetime.combine(atm_i['BASE_DT'], atm_i['TIME'])
        pr_i = atm_i['FUT']
        pr_o_i = atm_i['FUT_O']
        pr_h_i = atm_i['FUT_H']
        pr_l_i = atm_i['FUT_L']
        
        #장 시작 후, 일정 시간은 거래 skip
        mkt_o_tm = datetime.combine(atm_i['BASE_DT'], atm_i['TIME_O']) 
        if tm_i < mkt_o_tm + timedelta(seconds=strtg_begin_buf_tm):
            continue
        
        #장 종료 전, 청산
        mkt_c_tm = datetime.combine(atm_i['BASE_DT'], atm_i['TIME_C']) 
        if pos != 0 and mkt_c_tm - timedelta(seconds=strg_end_buf_tm) <= tm_i:
            
            pl_clr_i = np.sign(pos) * (pr_i - pr_ent)          
            fee_i = pr_i * fee_rt
            tot_pl = tot_pl + pl_clr_i - fee_i    
            
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_clr_i, 'FEE' : fee_i}])
            if len(exec) == 0:
                exec = exec_i
            else:
                exec = pd.concat([exec, exec_i])

            
            pos = 0              
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)            
            pr_ent = None # 진입가격
            pr_h = -np.inf
            pr_l = np.inf
            
            continue
        
        #장 종료 전, 일정 시간은 거래 skip 
        if mkt_c_tm - timedelta(seconds=strg_end_buf_tm) <= tm_i:
            continue
                
        pr_h = max(pr_h, pr_h_i)
        pr_l = min(pr_l, pr_l_i)
        
        opt_win_prv_i = atm_i['OPT_WIN_PRV']
        opt_win_i = atm_i['OPT_WIN']
        
        #1.1 손절 가격대이면, 손절(대응시간 변경, 가격 초기화, 손익 반영)
        pr_lc_i = 0
        if (pos > 0 and pr_l <= pr_ent - lc_dp):
            pr_lc_i = min(pr_ent - lc_dp, pr_o_i)
        elif (pos < 0 and pr_ent + lc_dp <= pr_h):
            pr_lc_i = max(pr_ent + lc_dp, pr_o_i)
            
        if pr_lc_i != 0: 
            
            pl_lc_i = -1 * abs(pr_lc_i - pr_ent)          
            fee_i = pl_lc_i * fee_rt
            tot_pl = tot_pl + pl_lc_i - fee_i
            
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'LC', 'PL' : pl_lc_i, 'FEE' : fee_i}])
            if len(exec) == 0:
                exec = exec_i
            else:
                exec = pd.concat([exec, exec_i])

            pos = 0              
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_lc_i
            pr_h = pr_lc_i
            pr_l = pr_lc_i
           
            continue    
        
        #1.2 익절 구간이면
        pr_pt_i = 0
        if (pos > 0 and pr_h >= pr_ent + pt_dp):
            pr_pt_i = max(pr_h, pr_ent + pt_dp) - pt_buf_dp
            
        elif (pos < 0 and pr_l <= pr_ent - pt_dp):
            pr_pt_i = min(pr_h, pr_ent - pt_dp) + pt_buf_dp
        
        #A. 익절 구간이고, 현재 가격이 익절 실행 가격이면
        if pr_pt_i != 0 and ((0 < pos and pr_l_i <= pr_pt_i) or (pos < 0 and pr_pt_i <= pr_h_i )) :
            pl_pt_i = abs(pr_pt_i - pr_ent)          
            fee_i = pl_pt_i * fee_rt
            tot_pl = tot_pl + pl_pt_i - fee_i
            
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'PT', 'PL' : pl_pt_i, 'FEE' : fee_i}])
            if len(exec) == 0:
                exec = exec_i
            else:
                exec = pd.concat([exec, exec_i])

            pos = 0              
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_pt_i
            pr_h = pr_pt_i
            pr_l = pr_pt_i
           
            continue    
          
            
        #2. 대응 유보 시간 이내 경우, skip 
        if tm_i <= react_hold_tm:
            #2.1 put/call cross 변경 되었으면 대응방향 변경
            if (pos_type == 'L' and (opt_win_prv_i == 'C' and opt_win_i == 'P')):
                pos_type = 'S'
            elif (pos_type == 'S' and (opt_win_prv_i == 'P' and opt_win_i == 'C')):
                pos_type = 'L'     
                
            continue
        
        #3. 대응 유보 시간 이후인 경우,      
        pl_i = 0   
        
        # CALL에서 PUT 우세로 전환되었고, 
        if opt_win_prv_i == 'C' and opt_win_i == 'P':
            pos_type = 'S'

            #현재 SHORT 포지션이면, 
            if pos < 0:
                #아무 작업하지 않음
                continue
            
            #현재 LONG 포지션이면, 
            if pos > 0:
                #매도를 통한 손익 실현
                pl_i = (pr_i - pr_ent)
                fee_i = pr_i * fee_rt
                tot_pl = tot_pl + pl_i - fee_i

                exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_i, 'FEE' : fee_i}])
                if len(exec) == 0:
                    exec = exec_i
                else:
                    exec = pd.concat([exec, exec_i])
                
            #SHORT 포지션 진입
            pos = -1

            fee_i = pr_i * fee_rt
            tot_pl = tot_pl - fee_i
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'ES', 'PL' : 0, 'FEE' : fee_i}])            
            if len(exec) == 0:
                exec = exec_i
            else:
                exec = pd.concat([exec, exec_i])

            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_i
            pr_h = pr_i
            pr_l = pr_i                     
                    
            continue
        
        # PUT에서 CALL 우세로 전환되었고, 
        if opt_win_prv_i == 'P' and opt_win_i == 'C':
            pos_type = 'L'
        
            #현재 LONG 포지션이면, 
            if pos > 0:
                #아무 작업하지 않음
                continue
            
            #현재 SHORT 포지션이면, 
            if pos < 0:
                #매수를 통한 손익 실현
                pl_i = (pr_ent - pr_i)                
                fee_i = pr_i * fee_rt
                tot_pl = tot_pl + pl_i - fee_i

                exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_i, 'FEE' : fee_i}])
                if len(exec) == 0:
                    exec = exec_i
                else:
                    exec = pd.concat([exec, exec_i])

                            
            #LONG 포지션 진입
            pos = 1
            
            fee_i = pr_i * fee_rt
            tot_pl = tot_pl - fee_i
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'EL', 'PL' : 0, 'FEE' : fee_i}])         

            if len(exec) == 0:
                exec = exec_i
            else:
                exec = pd.concat([exec, exec_i])
            
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_i
            pr_h = pr_i
            pr_l = pr_i 
                     
            continue       
    return [tot_pl, exec]

# strtg_begin_buf_tm = 6         # 전략 시작 시간 버퍼
# strg_end_buf_tm = 337.85   #전략 종료 시간 버퍼
# react_hold_dt = 5*60 # 대응유보시간
# ent_dp = 0.5         # 진입 가격 변동
# clr_dp = 0.5         # 대응 가격 변동
# lc_dp = 1        # 손절 가격 변동
# [pl, exec] = perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm)
# print(pl)
# print(exec)
# exec.to_csv("exec_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)


In [7]:
def _perf_pccrss(x, opt_atm):

    strtg_begin_buf_tm = x[0]         # 전략 시작 시간 버퍼
    strg_end_buf_tm = x[1]   #전략 종료 시간 버퍼
    react_hold_dt = x[2] # 대응유보시간
    ent_dp = 0.5         # 진입 가격 변동
    clr_dp = 0.5         # 대응 가격 변동
    lc_dp = x[3]         # 손절 가격 변동
    pt_dp = x[4]         # 익절 가격 변동
    pt_buf_dp = x[5]     # 익절 Buffer

    [pl, exec] = perf_pccrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, pt_dp, pt_buf_dp, opt_atm)
    
    return -1 * pl

def print_fun(x, opt_atm, convergence):
    x = np.round(x, 2)
    print("PL = %+.4f (strtg_begin_buf_tm = %+.2f, strg_end_buf_tm = %+.2f, react_hold_dt = %+.2f" \
        ", lc_dp = %+.2f, pt_dp = %+.2f, pt_buf_dp = %+.2f)" \
            % (-_perf_pccrss(x, opt_atm), x[0], x[1], x[2], x[3], x[4], x[5]))

def est_pccrss(opt_atm):

    bnds = ((0, 60*60), (0, 60*60), (0, 60*60), (0, 10), (0, 10), (0, 10))
    
    x0 = [3*60, 3*60, 1*60, 0.5, 1.5, 0.5]
    strategy = ['best1bin', 'best1exp', 'rand1exp', 'randtobest1exp', 'currenttobest1exp', 'best2exp', 'rand2exp'
                , 'randtobest1bin', 'currenttobest1bin', 'best2bin', 'rand2bin', 'rand1bin']
               
    x_init = x0
    
    opt_best = np.inf
    custom_args = [opt_atm]
    i = 0
    strategy_i = strategy[i]
    print("------------------- strategy : %s -------------------" % (strategy_i))
    opt = spo.differential_evolution(func = _perf_pccrss, args = custom_args, bounds = bnds, x0 = x_init, constraints=(), strategy = strategy_i
                                 , seed = i, disp = True, callback = lambda xk, convergence: print_fun(xk, est_opt_atm, convergence), polish = False, updating = 'deferred', workers = -1, maxiter = 10000)

    # while True:
        
    #     for i in range(len(strategy)):
    #         strategy_i = strategy[i]
    #         print("------------------- strategy : %s -------------------" % (strategy_i))
    #         opt = spo.differential_evolution(func = _perf_pccrss, args = custom_args, bounds = bnds, x0 = x_init, constraints=(), strategy = strategy_i
    #                                      , seed = i, disp = True, callback = lambda xk, convergence: print_fun(xk, est_opt_atm, convergence), polish = False, updating = 'deferred', workers = -1, maxiter = 3)
        
    #         x_init = opt.x
    #     if opt.fun >= opt_best:
    #         break
    #     opt_best = opt.fun

    return opt.x

In [8]:
def bktest_pccrss(est_opt_atm, perf_opt_atm):
    [strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, pt_dp, pt_buf_dp] = est_pccrss(est_opt_atm)
    [pl, exec] = perf_pccrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, pt_dp, pt_buf_dp, perf_opt_atm)
    
    param = {'strtg_begin_buf_tm' : strtg_begin_buf_tm, 'strg_end_buf_tm' : strg_end_buf_tm, 
             'react_hold_dt' : react_hold_dt, 
             'lc_dp' : lc_dp, 
             'pt_cp' : pt_dp, 'pt_buf_dp' : pt_buf_dp 
            }
    return {'PL' : pl, 'EXEC' : exec, 'PARAM' : param}

In [10]:
d_dir = './data(mini)'
base_ym = list(range(202001, 202013))           \
            + list(range(202101, 202113))       \
            + list(range(202201, 202213))       \
            + list(range(202301, 202313))       \
            + list(range(202401, 202409))         
             
base_ym.sort()
base_ym_begin = base_ym[0]
base_ym_end = base_ym[-1]

opt_atm = pd.DataFrame()
copt = pd.DataFrame()
popt = pd.DataFrame()
fut = pd.DataFrame()
idx = pd.DataFrame()

for base_ym_i in base_ym:
    print(base_ym_i)
    [opt_atm_i, copt_i, popt_i, fut_i] = get_opt_atm(base_ym_i, d_dir)
    opt_atm = pd.concat([opt_atm, opt_atm_i])
    copt = pd.concat([copt, copt_i])
    popt = pd.concat([popt, popt_i])
    fut = pd.concat([fut, fut_i])
    
opt_atm.to_csv("ATM_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)

202001
202002
202003
202004
202005
202006


KeyboardInterrupt: 

In [ ]:
base_ym = np.sort(np.unique(opt_atm['BASE_YM']))
PERF_M = 1
n_ym = len(base_ym)
    
for est_m in [1, 3, 6, 12]:
    
    pl = pd.DataFrame()
    exec = pd.DataFrame()
    param = pd.DataFrame()
    
    for i in range(n_ym - est_m - PERF_M + 1):
        data_ym = base_ym[range(i, i + est_m + PERF_M)];
        [est_ym, perf_ym] = np.split(data_ym, [est_m]);
        
        est_opt_atm = opt_atm[opt_atm['BASE_YM'].isin(est_ym)]
        perf_opt_atm = opt_atm[opt_atm['BASE_YM'].isin(perf_ym)]
        
        ret = bktest_pccrss(est_opt_atm, perf_opt_atm)

        ret_pl = pd.DataFrame({'PL' : [ret['PL']]})
        ret_pl.insert(0, 'PERF_YM', perf_ym[0])
        pl = pd.concat([pl, ret_pl])

        ret_exec = pd.DataFrame(ret['EXEC'])
        ret_exec.insert(0, 'PERF_YM', perf_ym[0])
        exec = pd.concat([exec, ret_exec])

        ret_param = pd.DataFrame([ret['PARAM']])
        ret_param.insert(0, 'PERF_YM', perf_ym[0])
        param = pd.concat([param, ret_param])

        print(pl)
        
    pl.to_csv("pl_%d.csv" % (est_m), index=False)
    exec.to_csv("exec_%d.csv" % (est_m), index=False)
    param.to_csv("param_%d.csv" % (est_m), index=False)
